## Implementação prática de um algoritmo de Regressão Logística
### Fazendo uso da função de ativação sigmoidal (logistic)

$$logistic= \frac{1}{(1 + e^{-z})}$$

Tal que $z$ será:

$$ z = (a_1 \cdot x_1 + a_2 \cdot x_2 + ... + a_n \cdot x_n) + b $$

In [37]:
#Importação das bibliotecas necessárias
import numpy as np
import zipfile
import pandas as pd

### Definição do modelo e seus métodos:
* Precisão
* Acurácia
* Acurácia Balanceada
* Teste (Prediction)
* Treino (Fit)

In [38]:
class BinaryLogisticRegressor:
    
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None
        
    #Definicao da função de ativação       
    def _sigmoid(self, x):
        x = np.array(x)
        x=x.reshape(-1,1)
        x = x.astype(float)
        #Função sigmoide
        return 1 / (1 + np.exp(-x))
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Inicialização dos pesos
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # Gradiente descendente
        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)
            
            # Calculo do gradiente
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            
            # Atualização dos pesos
            self.weights = self.weights.reshape(-1,1)
            self.weights = self.weights.astype(float)
            dw= dw.reshape(-1,1)
            dw = dw.astype(float)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
    
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        #Aplicacao da função de ativação para retornar a classe predita
        y_predicted = self._sigmoid(linear_model)
        #Classificação da classe predita de forma binária (0 ou 1)
        y_predicted_cls = [1 if i < 0.42 else 0 for i in y_predicted]
        return y_predicted_cls
    

In [39]:
#Definindo um classe para escalonamento dos dados 
#Para os seguintes metodos: MinMax, Standard e Normal
class Scaler:
        
        def __init__(self, method):
            self.method = method
        
        def fit(self, X):
            if self.method == 'minmax':
                self.min = np.min(X, axis=0)
                self.max = np.max(X, axis=0)
            elif self.method == 'std':
                self.mean = np.mean(X, axis=0)
                self.std = np.std(X, axis=0)
            elif self.method == 'normal':
                self.mean = np.mean(X, axis=0)
                self.std = np.std(X, axis=0)
            else:
                raise Exception('Método inválido ou não implementado')
        
        def transform(self, X):
            if self.method == 'minmax':
                return (X - self.min) / (self.max - self.min)
            elif self.method == 'std':
                return (X - self.mean) / self.std
            elif self.method == 'normal':
                return (X - self.mean) / self.std
            else:
                raise Exception('Método inválido ou não implementado')


In [40]:
class Metrics:
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.positive= np.sum(y_true == 1)
        self.negative= np.sum(y_true == 0)
        
        false_positive = 0
        false_negative = 0
        true_positives = 0
        true_negatives = 0
        
        for i in range (len(y_true)):
            if y_pred[i] == 1 and y_true[i] == 1:
                true_positive += 1
            elif y_pred[i] == 1 and y_true[i] == 0:
                false_positive += 1
            elif y_pred[i] == 0 and y_true[i] == 1:
                false_negative += 1
            elif y_pred[i] == 0 and y_true[i] == 0:
                true_negatives += 1
        
        self.false_positive = false_positive
        self.false_negative = false_negative
        self.true_positives = true_positives
        self.true_negatives = true_negatives
        
        
        
    def accuracy(self):
        accuracy = np.sum(self.y_true == self.y_pred) / len(self.y_true)
        return accuracy
    
    def balanced_accuracy(self):
        balanced_accuracy = (self.true_positives / self.positive + self.true_negatives / self.negative) / 2
        return balanced_accuracy
    
    def sensitivity(self):
        sensitivity = self.true_positives / self.positive
        return sensitivity
    
    def specificity(self):
        specificity = self.true_negatives / self.negative
        return specificity

            

In [41]:
#Implementaçao de classe para separar os dados em treino e teste dado um modelo de separação
#Kfold, TrainTestSplit 

class Split:
    def __init__(self, method):
        self.method = method
    
    def kfold(self, X, y, k):
        n_samples = X.shape[0]
        n_samples_per_fold = n_samples // k
        X_folds = []
        y_folds = []
        for i in range(k):
            X_folds.append(X[i * n_samples_per_fold : (i + 1) * n_samples_per_fold])
            y_folds.append(y[i * n_samples_per_fold : (i + 1) * n_samples_per_fold])
        return X_folds, y_folds

    def train_test_split(self, X, y, test_size):
        n_samples = X.shape[0]
        n_samples_test = int(n_samples * test_size)
        n_samples_train = n_samples - n_samples_test
        X_train = X[:n_samples_train]
        y_train = y[:n_samples_train]
        X_test = X[n_samples_train:]
        y_test = y[n_samples_train:]
        return X_train, X_test, y_train, y_test

In [42]:
#Importando o dataset
zf = zipfile.ZipFile('archive.zip')
data = pd.read_csv(zf.open('data.csv'))

In [43]:
#Visualizando as features presentes no modelo
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [44]:
#Visualizando as features presentes no modelo
data.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [45]:
data.shape

(569, 33)

In [46]:
#Selecionando as features que serão utilizadas no modelo
X = data.values[:, 2]
y = data.values[:, 1]

In [47]:
#Visualizando as classes presentes no dataset
np.unique(y)

array(['B', 'M'], dtype=object)

In [48]:
#Convertendo as classes para valores binários
y = np.where(y == 'M', 1, 0)

In [49]:
#Verificando a conversão
np.unique(y)

array([0, 1])

In [50]:
#Convertendo os dados para vetores do tipo np.array
X = np.array(X)
X=X.reshape(-1,1)
y = np.array(y)
y=y.reshape(-1,1)

In [51]:
#Instanciando o classificador e o escalonador
clf = BinaryLogisticRegressor()
sc= Scaler('minmax')

In [52]:
#Escalonando os dados
sc.fit(X)
X = sc.transform(X)

print(min(X), max(X))

[0.0] [1.0]


In [53]:
#Instanciando o objeto para separação dos dados
#Utilizando o TrainTestSplit

split = Split('train_test_split')

#Separando os dados em treino e teste com 20% dos dados para teste
X_train, X_test, y_train, y_test = split.train_test_split(X, y, 0.2)



In [54]:
#Treinando o modelo
clf.fit(X_train, y_train)

In [55]:
#Realizando a predição
y_pred = clf.predict(X_test)
y_pred = np.array(y_pred)



In [56]:
m=Metrics(y_test, y_pred)

print('Acurácia: ', m.accuracy())
print('Acurácia Balanceada: ', m.balanced_accuracy())
print('Sensibilidade: ', m.sensitivity())
print('Especificidade: ', m.specificity())

print('Falso Positivo: ', m.false_positive)
print('Falso Negativo: ', m.false_negative)
print('Verdadeiro Positivo: ', m.true_positives)
print('Verdadeiro Negativo: ', m.true_negatives)
print('Positivo: ', m.positive)
print('Negativo: ', m.negative)



Acurácia:  64.86725663716814
Acurácia Balanceada:  0.26436781609195403
Sensibilidade:  0.0
Especificidade:  0.5287356321839081
Falso Positivo:  41
Falso Negativo:  26
Verdadeiro Positivo:  0
Verdadeiro Negativo:  46
Positivo:  26
Negativo:  87
